# import  library 

In [ ]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from keras.models import Model
from keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#  load data 

In [ ]:

data = pd.read_csv('../input/kc-house-data/kc_house_data.csv')
data.head()

In [ ]:
data = data[ ['sqft_lot', 'floors', 'price' ,'bedrooms' ,'bathrooms' ,'grade' ,'sqft_living15' ,'sqft_lot15' ,'zipcode'] ]
data['sqft_lot'].hist(bins=50)
data.describe().astype(int)

# logarithme normalization 

In [ ]:

data['log_sqft_lot'] = np.log(1+ data['sqft_lot'])
data['log_price'] = np.log(1+ data['price'])
data['log_bedrooms'] = np.log(1+ data['bedrooms'])
data['log_sqft_living15'] = np.log(1+ data['sqft_living15'])
data['log_sqft_lot15'] = np.log(1+ data['sqft_lot15'])
data_log= data[['log_sqft_lot' ,'log_price' ,'floors','log_bedrooms','bathrooms','grade','log_sqft_living15','log_sqft_lot15' ,'zipcode']]
data_log['log_sqft_lot'].hist(bins=50)
data_log.describe().astype(int)


In [ ]:

X = data[ ['log_sqft_lot', 'floors','log_bedrooms','bathrooms','grade' ,'log_sqft_living15','log_sqft_lot15'] ].values 
Y = data[ 'log_price' ].values  

X_train = X[:20000]
Y_train = Y[:20000]
X_test = X[20000:]
Y_test = Y[20000:]



# model data with  logarithme normalization 

In [ ]:
inpt = layers.Input( shape=(7,) )
x    = layers.Dense(100) ( inpt )
actv = layers.Activation('relu') (x)
output = layers.Dense(1) (actv)

model = Model( inpt, output )
model.summary()

In [ ]:
opt = SGD() 
model.compile( optimizer=opt, loss='mse' )
model.fit( X_train, Y_train, epochs=20 )

In [ ]:
Y=np.exp(Y_test)-1
print(Y[:6])

In [ ]:
predictions = model.predict(X_test)
pred=np.exp(predictions)-1
print(pred[:6])


# standard normalization 

In [ ]:
data_X = data[ ['sqft_lot', 'floors' ,'bedrooms' ,'bathrooms' ,'grade' ,'sqft_living15' ,'sqft_lot15'] ]
data_Y = data[['price']]


# define standard scaler
X_scaler_standard = StandardScaler()
Y_scaler_standard = StandardScaler()
# transform data
X_scaler_standard.fit(data_X)
Y_scaler_standard.fit(data_Y)

data_X_scaler = X_scaler_standard.fit_transform(data_X)
data_Y_scaler = Y_scaler_standard.fit_transform(data_Y)

X_train = data_X_scaler[:20000]
Y_train = data_Y_scaler[:20000]

X_test = data_X_scaler[20000:]
Y_test = data_Y_scaler[20000:]



# model data standard normalization 

In [ ]:
inpt = layers.Input( shape=(7,) )
x    = layers.Dense(100) ( inpt )
actv = layers.Activation('relu') (x)
output = layers.Dense(1) (actv)

model_standard = Model( inpt, output )
model_standard.summary()
model_standard.compile( optimizer=opt, loss='mse' )
model_standard.fit( X_train, Y_train, epochs=30 )

In [ ]:
# -----------------
predictions_standard = model_standard.predict(X_test)
#predi = X_scaler.inverse_transform(predictions_standard)
predictions_standard=Y_scaler_standard.inverse_transform(predictions_standard)
print("predictions_ standard  : " ,predictions_standard[:6].T.astype(int))


In [ ]:

data_Y[20000:20006]

# Min max normalization

In [ ]:
# define standard scaler
X_scaler_minmax = MinMaxScaler()
Y_scaler_minmax = MinMaxScaler()
# transform data
X_scaler_minmax.fit(data_X)
Y_scaler_minmax.fit(data_Y)

data_X_minmax = X_scaler_minmax.fit_transform(data_X)
data_Y_minmax = Y_scaler_minmax.fit_transform(data_Y)

X_train = data_X_minmax[:20000]
Y_train = data_Y_minmax[:20000]

X_test = data_X_minmax[20000:]
Y_test = data_Y_minmax[20000:]



# model : data normalization min max

In [ ]:
inpt = layers.Input( shape=(7,) )
x    = layers.Dense(100) ( inpt )
actv = layers.Activation('relu') (x)
output = layers.Dense(1) (actv)

model_minmax = Model( inpt, output )
model_minmax.summary()
model_minmax.compile( optimizer=opt, loss='mse' )
model_minmax.fit( X_train, Y_train, epochs=30 )

In [ ]:
# -----------------
predictions_minmax = model_minmax.predict(X_test)

predictions_minmax=Y_scaler_minmax.inverse_transform(predictions_minmax)
print("predictions   : " ,predictions_minmax[:6].T.astype(int))

In [ ]:
data_Y[20000:20006]

# Now we add zip code 

In [ ]:
zipcode_dummies = pd.get_dummies(data['zipcode'])
zipcode_dummies

# concatenate with array data_X_minmax 

In [ ]:
data_X_zipcode_minmax = np.concatenate((data_X_minmax, zipcode_dummies), axis=1)
data_X_zipcode_minmax.shape

In [ ]:
X_train_zipcode_minmax = data_X_zipcode_minmax[:20000]
Y_train_zipcode_minmax  = data_Y_minmax[:20000]

X_test_zipcode_minmax  = data_X_zipcode_minmax[20000:]
Y_test_zipcode_minmax  = data_Y_minmax[20000:]


# model and  data with zipcode and max min normalisation 

In [ ]:

inpt = layers.Input( shape=(77,) )
x    = layers.Dense(200) ( inpt )
actv = layers.Activation('relu') (x)
output = layers.Dense(1) (actv)

model_zipcode_minmax = Model( inpt, output )
model_zipcode_minmax.summary()
model_zipcode_minmax.compile( optimizer=opt, loss='mse' )
model_zipcode_minmax.fit( X_train_zipcode_minmax, Y_train_zipcode_minmax, epochs=30 )

In [ ]:
predictions_zipcode_minmax = model_zipcode_minmax.predict(X_test_zipcode_minmax)

predictions_zipcode_minmax=Y_scaler_minmax.inverse_transform(predictions_zipcode_minmax)

print("predictions   : " ,predictions_zipcode_minmax[:6].T.astype(int))

In [ ]:
data_Y[20000:20006]